# Transformaciones

Los DStreams en Spark Streaming se tratan de forma similar a los RDDs cuando trabajamos con Spark. Existen muchas opciones para realizar transformaciones sobre los datos:

| Transformation        | Meaning         |
| ------------------------------ |:-------------|
| **map**(func)      | Se aplica la función func a cada elemento de DStream. Devuelve un nuevo DStream.    |
| **flatMap**(func)	| Similar a map, pero por cada elemento de entrada se devuelven de 0 a n elementos de salida.    |
| **filter**(func)	| Nuevo DStream con los elementos del DStream original para los que func devuelva True    |
| **repartition**(numPartitions)	| Modifica el nivel de paralelismo de un DStream.    |
| **union**(otherStream)	| Nuevo DStream con la unión del actual y de otherStream. |
| **count**()	| Nuevo DStream con RDDs simples con el número de elementos de cada RDD en el DStream original  |
| **reduce**(func)	| Agregación de elementos de cada RDD del DStream original para generar un nuevo DStream con RDDs simples. La función func ha de ser conmutativa y asociativa, recibir dos argumentos y devolver uno. |
| **countByValue**()	| Sobre un DStream de elementos de tipo K, devuelve la frecuencia de cada elemento del DStream en formato (K, Long). |
| **reduceByKey**(func, [numTasks])	|Sobre un DStream de elementos clave-valor (K,V), devuelve la agregación de todos los valores con la misma clave, según la función func. Se puede especificar el número de tareas paralelas de forma opcional (por defecto 2). |
| **join**(otherStream, [numTasks])	| Sobre dos DStreams (original y otherStream) de clave-valor (K,V) y (K,W), devuelve un DStream (K, (V,W)), sólo para aquéllas claves que aparezcan en ambos DStreams. Número de tareas paralelas opcional. |
| **cogroup**(otherStream, [numTasks])	| Sobre dos DStreams (original y otherStream) de clave-valor (K,V) y (K,W), devuelve un nuevo DStream con tuplas (K, Seq[V], Seq[W]), para todas las claves que aparezcan al menos en uno de los DStreams. Número de tareas paralelas opcional. |



# Demo

In [ ]:
import findspark
# Importante: Modificar la ruta para que apunte al HOME de Spark
findspark.init('/opt/spark')

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pprint import pprint

In [ ]:
#Se especifica el contexto Spark y el contexto Streaming definiendo la duración del batch (en este caso, 4 segundos)
sc = SparkContext(master="local[2]", appName="EjemploTransformaciones")
scc = StreamingContext(sc, 4)

### Introducción de datos
Se va a utilizar un socket para introducir datos. Dicho socket estará escuchando en el puerto 9999 de localhost. Para la introducción de datos se puede abrir una ventana de comandos y ejecutar el comando "nc -lk 9999", que abre el puerto y lo mantiene abierto, para a continuación pegar los mensajes que se quieren enviar. Si no existe el comando, deberemos instalar el programa "netcat" --> "sudo apt-get install netcat"

In [ ]:
lines = scc.socketTextStream("localhost", 9999)

Se van a probar cuatro transformaciones distintas:
- En la primera, se eliminan signos de puntuación del texto introducido, y se pasa a minúsculas.
- En la segunda, se utiliza un flatMap para que cada elemento del RDD dentro del DStream sea una palabra.
- En la tercera, se filtran aquéllas palabras que sean números del 0 al 9.
- En la cuarta, se realiza un conteo de la frecuencia de cada palabra, para cada batch de datos.

Se puede probar a introducir el siguiente texto (varias líneas):
   
Esta es la línea 1.  
Esta es la línea 2.  
Esta es la línea 3.   
Otra línea más, esta sin número.



In [ ]:
transformations1 = lines.map(lambda x: x.replace(',',' ').replace('?',' ').replace(':',' ').replace('.',' ').replace('-',' ').lower())
transformations1.pprint()
transformations2 = transformations1.flatMap(lambda line: line.split(" "))
transformations2.pprint()
stringNumbers = {'1','2','3','4','5','6','7','8','9','0'}
transformations3 = transformations2.filter(lambda x: x not in stringNumbers)
transformations3.pprint()
transformations4 = transformations3.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
transformations4.pprint()


Mediante start() ejecutamos el programa, y lo detenemos cuando queramos con stop().

In [ ]:
scc.start()

In [ ]:
scc.stop(stopSparkContext=True, stopGraceFully=True)